# Assignment 1
Name: Mykhailovskyi Oleksandr  
CSCI E-222 Introduction to Natural Language Processing

## Setup

In [1]:
import os
# from pathlib import Path

token = None

# Colab Secrets
try:
    from google.colab import userdata  # type: ignore

    token = (userdata.get("HF_TOKEN") or "").strip()
except Exception:
    token = None

# # Local File
# if not token:
#     token_path = Path("hugging_face_token.txt")
#     if token_path.exists():
#         token = token_path.read_text(encoding="utf-8").strip()

if token:
    os.environ["HF_TOKEN"] = token
    os.environ["HUGGINGFACE_HUB_TOKEN"] = token
    print("Hugging Face API token loaded successfully.")
else:
    print("No Hugging Face API token found.")


Hugging Face API token loaded successfully.


## Problem 1 (25 points)

In this problem, you will work with a pre-trained autoregressive language model and see how it predicts the next token.

Please use the decoder‑only text‑generation model gpt2 from the Hugging Face transformers library, and use it only in inference mode (no training or fine‑tuning).

(a) Install and import the necessary libraries. At a minimum, your code needs:
    <pre>
    import torch
    from transformers import AutoTokenizer, AutoModelForCausalLM
    </pre>

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


(b) Load the tokenizer and model for gpt2:
    <pre>
    tokenizer = AutoTokenizer.from_pretrained(“gpt2”)
    model = AutoModelForCausalLM.from_pretrained(“gpt2”)
    model.eval()
    </pre>

In [3]:
# load the tokenizer and model for gpt2
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.eval()

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

(c) In your report, briefly state that you used the model gpt2 and indicate the approximate number of parameters used (an order-of-magnitude description is sufficient).

<b>Answer</b>: gpt2 model has 124,439,808 parameters

In [4]:
print("Model & tokenizer for GPT2 model loaded successfully, number of parameters: ", model.num_parameters())

Model & tokenizer for GPT2 model loaded successfully, number of parameters:  124439808


(d) For the following prompt:
    <pre>
    So long and thanks for
    </pre>
    identify the top 5 candidate tokens and their probabilities according to gpt2 and then find the most likely next token.

<b>Answer</b>:  
| Prob-ty | Token |
| ------- | ----- |
| 0.1581  | ' reading'  
| 0.1283  | ' the'  
| 0.1238  | ' your'    
| 0.0732  | ' all'  
| 0.0323  | ' listening'  

The most probable next token is 'reading' with probability of 15.8%.

(e) In your report, show the code you used and list the top 5 tokens and their probabilities in a small table or clearly formatted list. Briefly comment on whether the most likely token seems reasonable, given the prompt.

<b>Answer</b>: The most likely next token seems reasonable, given a preceding sentence, let's take a look at the whole sentence with the predicted token - 'So long and thanks for reading'. This sentence seems reasonable, I can imagine it being at the end of an article.

In [5]:
prompt = "So long and thanks for"

inputs = tokenizer(prompt, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    # shape of below is (batch_size, sequence_length, vocab_size)
    # here (1, seq_len, 50,257), gpt2 vocab size is 50,257
    # because the model will predict for each token in a given sequence as if the sentence ended there
    logits = outputs.logits

next_token_logits = logits[0, -1, :] # shape (vocab_size,)
probs = torch.softmax(next_token_logits, dim=-1)

top_probs, top_indices = torch.topk(probs, k=5)
top_tokens = [tokenizer.decode([idx]) for idx in top_indices]

top_5_predictions = list(zip(top_tokens, top_probs.tolist()))

print("Top 5 predicted next tokens with probabilities:")
for token, prob in top_5_predictions:
    print(f"Prob-ty: {prob:.4f} | Token: '{token}'")

print("\nMost likely next token:", top_5_predictions[0][0])

Top 5 predicted next tokens with probabilities:
Prob-ty: 0.1581 | Token: ' reading'
Prob-ty: 0.1283 | Token: ' the'
Prob-ty: 0.1238 | Token: ' your'
Prob-ty: 0.0732 | Token: ' all'
Prob-ty: 0.0323 | Token: ' listening'

Most likely next token:  reading


## Problem 2
In this problem, you will expand the application of gpt2 to the completion of a sentence.

(a) Using the same model gpt2 and the following prompt:
<pre>
The only true wisdom is
</pre>
generate a continuation of about 15-20 new tokens using greedy decoding (i.e., always pick the most probable next token at each step). One simple way is:
<pre>
inputs = tokenizer(prompt, return_tensors=”pt”)
outputs = model.generate(
**inputs,
max_new_tokens=20,
    		do_sample=False   # greedy decoding
)
Completion = tokenizer.decode(
outputs[0],
skip_special_tokens=True
)
</pre>


In [6]:
prompt = "The only true wisdom is"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=20, do_sample=False) # greedy decoding
completion = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Completion:", completion)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Completion: The only true wisdom is that we should not be afraid of the consequences of our actions.

The only true wisdom is


(b) In your report, show the full completed text (original prompt plus generated continuation). Briefly answer (3-5 sentences): Does the continuation look grammatical and locally coherent? Does it make sense? How does it compare with the famous Socrates’ version?

<b>Answer</b>:  
>"The only true wisdom is in knowing that you know nothing."    
>—Socrates

Compare the original above to the gpt2 version:  
>"The only true wisdom is that we should not be afraid of the consequences of our actions."    
>—gpt2

Given, a prompt "The only true wisdom is" - gpt2 managed to produce a coherent phrase, which makes sense, and could be falsely attributed to Socrates. Personally, I find the original version more close to my heart, but that could also be attributed towards bias in knowing that the latter is machine produced output, and the former is actual Socrates' quote. Nevertheless, the gpt2 phrase is also inspiring and makes sense.


## Problem 3
In this problem, you will explore few‑shot and zero‑shot prompting for a simple sentiment classification task using the same pre‑trained language model as in Problem 1.
Please continue to use the decoder-only model gpt2 from the Hugging Face transformers library, in inference mode only (no training or fine‑tuning).

(a) We will construct a few‑shot prompt for restaurant‑review sentiment classification. For the following prompt:
<pre>
Review: The food was delicious and the service was very friendly.
Sentiment: positive
Review: The dishes were cold and the staff were rude.
Sentiment: negative
Review: I waited an hour for my meal and it was still undercooked.
Sentiment:
</pre>

identify the top 10 candidate tokens and their probabilities according to gpt2 and then determine the most likely next token. Note: When you build this prompt in Python, please preserve the line breaks using \n.

In your report, list the top 10 tokens and their probabilities in a small table or clearly formatted list. Briefly comment on whether the most likely token seems reasonable.

<b>Answer</b>:  
| Prob-ty | Token |
| ------- | ----- |
| 0.4828 | ' negative' |
| 0.2952 | ' positive' |
| 0.0163 | ' very' |
| 0.0133 | ' bad' |
| 0.0116 | ' good' |
| 0.0069 | ' not' |
| 0.0040 | ' 1' |
| 0.0031 | ' neutral' |
| 0.0030 | ' Negative' |
| 0.0028 | ' 3' |

Most probable next token is ' negative', which corresponds to the sentiment of the review. The model correctly assigns the highest probability to this token: 48%.


In [7]:
prompt = """Review: The food was delicious and the service was very friendly.
Sentiment: positive
Review: The dishes were cold and the staff were rude.
Sentiment: negative
Review: I waited an hour for my meal and it was still undercooked.
Sentiment:""" # will preserve line breaks

inputs = tokenizer(prompt, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

next_token_logits = logits[0, -1, :]
probs = torch.softmax(next_token_logits, dim=-1)

top_probs, top_indices = torch.topk(probs, k=10)
top_tokens = [tokenizer.decode([idx]) for idx in top_indices]

top_10_preds = list(zip(top_tokens, top_probs.tolist()))

print("Top 10 predicted next tokens with probabilities:")
for token, prob in top_10_preds:
    print(f"Prob-ty: {prob:.4f} | Token: '{token}'")

print("\nMost likely next token:", top_10_preds[0][0])

Top 10 predicted next tokens with probabilities:
Prob-ty: 0.4828 | Token: ' negative'
Prob-ty: 0.2952 | Token: ' positive'
Prob-ty: 0.0163 | Token: ' very'
Prob-ty: 0.0133 | Token: ' bad'
Prob-ty: 0.0116 | Token: ' good'
Prob-ty: 0.0069 | Token: ' not'
Prob-ty: 0.0040 | Token: ' 1'
Prob-ty: 0.0031 | Token: ' neutral'
Prob-ty: 0.0030 | Token: ' Negative'
Prob-ty: 0.0028 | Token: ' 3'

Most likely next token:  negative


(b) Design a new prompt by making small changes to the text above so that, for the last review, the most likely next token according to gpt2 is either the token good or the token bad. In your report, show the new prompt and the corresponding top‑10 next‑token distribution.

<b>Answer</b>:
| Prob-ty | Token |
| ------- | ----- |
| 0.3266 | ' bad' |
| 0.2492 | ' good' |
| 0.0241 | ' '' |
| 0.0218 | ' not' |
| 0.0176 | ' very' |
| 0.0172 | ' poor' |
| 0.0124 | ' ok' |
| 0.0115 | ' terrible' |
| 0.0113 | ' great' |
| 0.0083 | ' positive' |

As can be seen above, the model still correctly assigns the biggest probability to the negative sentiment token. Although, the absolute value of probability for the word 'bad' is less than that for the word 'negative'. We could reinforce this by, for example, introducing additional instructions before the examples, such as
>The following are sentiment recognition examples for restaurant reviews.
>Sentiment is either 'good' for positive reviews or 'bad' for negative reviews.
>When predicting sentiment, only output the word, don't use quotes, punctuation, or any other characters.

In [8]:
prompt = """Review: The food was delicious and the service was very friendly.
Sentiment: good
Review: The dishes were cold and the staff were rude.
Sentiment: bad
Review: I waited an hour for my meal and it was still undercooked.
Sentiment:""" # will preserve line breaks

inputs = tokenizer(prompt, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

next_token_logits = logits[0, -1, :]
probs = torch.softmax(next_token_logits, dim=-1)

top_probs, top_indices = torch.topk(probs, k=10)
top_tokens = [tokenizer.decode([idx]) for idx in top_indices]

top_10_preds = list(zip(top_tokens, top_probs.tolist()))

print("Top 10 predicted next tokens with probabilities:")
for token, prob in top_10_preds:
    print(f"Prob-ty: {prob:.4f} | Token: '{token}'")

print("\nMost likely next token:", top_10_preds[0][0])

Top 10 predicted next tokens with probabilities:
Prob-ty: 0.3373 | Token: ' bad'
Prob-ty: 0.0808 | Token: ' good'
Prob-ty: 0.0325 | Token: ' terrible'
Prob-ty: 0.0317 | Token: ' very'
Prob-ty: 0.0310 | Token: ' poor'
Prob-ty: 0.0291 | Token: ' not'
Prob-ty: 0.0152 | Token: ' ok'
Prob-ty: 0.0143 | Token: ' I'
Prob-ty: 0.0142 | Token: ' horrible'
Prob-ty: 0.0140 | Token: ' great'

Most likely next token:  bad


(c) Now design a zero‑shot prompt (no labeled examples, only an instruction) that asks gpt2 to classify the single review
<pre>
I waited an hour for my meal and it was still undercooked.
</pre>
as either positive or negative. In your report, show the exact zero‑shot prompt you used and list the top 10 tokens and their probabilities. Comment on whether the most likely token corresponds to the correct sentiment for this review. Briefly comment on how this zero‑shot result compares to your few‑shot results from part (a).

<b>Answer</b>:
| Prob-ty | Token |
| ------- | ----- |
| 0.1944 | 'good' |
| 0.1207 | 'Good' |
| 0.0881 | 'I' |
| 0.0205 | 'bad' |
| 0.0150 | 'It' |
| 0.0136 | 'Great' |
| 0.0133 | 'This' |
| 0.0113 | 'You' |
| 0.0113 | 'not' |
| 0.0106 | 'Not' |

The most probable next token is 'good' with probability of 19.4%, the correct next token would be 'bad' with probability of 2%. The model fails to identify the correct sentiment in this zero-shot prompt, unlike for the examples of few-shot prompts above.

In [9]:
prompt = """The following are sentiment recognition examples for restaurant reviews.
Sentiment is either 'good' for positive reviews or 'bad' for negative reviews.
Review: I waited an hour for my meal and it was still undercooked.
Sentiment: '"""

inputs = tokenizer(prompt, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

next_token_logits = logits[0, -1, :]

probs = torch.softmax(next_token_logits, dim=-1)

top_probs, top_indices = torch.topk(probs, k=10)
top_tokens = [tokenizer.decode([idx]) for idx in top_indices]

top_10_preds = list(zip(top_tokens, top_probs.tolist()))
print("Top 10 predicted next tokens with probabilities:")
for token, prob in top_10_preds:
    print(f"Prob-ty: {prob:.4f} | Token: '{token}'")

print("\nMost likely next token:", top_10_preds[0][0])

Top 10 predicted next tokens with probabilities:
Prob-ty: 0.1944 | Token: 'good'
Prob-ty: 0.1207 | Token: 'Good'
Prob-ty: 0.0881 | Token: 'I'
Prob-ty: 0.0205 | Token: 'bad'
Prob-ty: 0.0150 | Token: 'It'
Prob-ty: 0.0136 | Token: 'Great'
Prob-ty: 0.0133 | Token: 'This'
Prob-ty: 0.0113 | Token: 'You'
Prob-ty: 0.0113 | Token: 'not'
Prob-ty: 0.0106 | Token: 'Not'

Most likely next token: good


## Sources
- Jurafsky, D., and Martin, J. H. "Speech and Language Processing: An Introduction to Natural Language Processing, Computational Linguistics, and Speech Recognition with Language Models" (3rd ed. draft). https://web.stanford.edu/~jurafsky/slp3/